In [12]:
from models import WorkerOrm, ResumeOrm, Workload
from schema import ResumeDTO, ResumeRelDTO, WorkerDTO, WorkersRelDTO
from database import session_factory, sync_engine

from sqlalchemy import select, func, and_, or_, Integer
from sqlalchemy.orm import selectinload

*Без relationship*

In [15]:
with session_factory() as session:
    query = (
        select(WorkerOrm)
    )
    res = session.execute(query)
    result_orm = res.scalars().all()

    print(f"{result_orm=}")

result_dto = [WorkerDTO.model_validate(row, from_attributes=True) for row in result_orm]
for item in result_dto: print(f"{item=}")


result_orm=[<WorkerOrm, id=2, username=John>, <WorkerOrm, id=1, username=Michanya>, <WorkerOrm, id=3, username=Artem>, <WorkerOrm, id=4, username=Roman>, <WorkerOrm, id=5, username=Petr>]
item=WorkerDTO(username='John', id=2)
item=WorkerDTO(username='Michanya', id=1)
item=WorkerDTO(username='Artem', id=3)
item=WorkerDTO(username='Roman', id=4)
item=WorkerDTO(username='Petr', id=5)


***С relationship***

In [18]:
with session_factory() as session:
    query = (
        select(WorkerOrm)
        .options(selectinload(WorkerOrm.resumes))
        .limit(2)
    )
    res = session.execute(query)
    result_orm = res.scalars().all()
print(f"{result_orm=}")
result_dto = [WorkersRelDTO.model_validate(row, from_attributes=True) for row in result_orm]
for item in result_dto: print(item)


result_orm=[<WorkerOrm, id=2, username=John>, <WorkerOrm, id=1, username=Michanya>]
username='John' id=2 resumes=[ResumeDTO(title='Python Data Engineer', compensation=250000, workload=<Workload.parttime: 'parttime'>, worker_id=2, id=3, created_at=datetime.datetime(2024, 5, 3, 11, 33, 38, 242699), updated_at=datetime.datetime(2024, 5, 3, 11, 33, 38, 242699)), ResumeDTO(title='Data Scientist', compensation=300000, workload=<Workload.fulltime: 'fulltime'>, worker_id=2, id=4, created_at=datetime.datetime(2024, 5, 3, 11, 33, 38, 242699), updated_at=datetime.datetime(2024, 5, 3, 11, 33, 38, 242699))]
username='Michanya' id=1 resumes=[ResumeDTO(title='Python Junior Developer', compensation=50000, workload=<Workload.fulltime: 'fulltime'>, worker_id=1, id=1, created_at=datetime.datetime(2024, 5, 3, 11, 33, 38, 242699), updated_at=datetime.datetime(2024, 5, 3, 11, 33, 38, 242699)), ResumeDTO(title='Python Разработчик', compensation=150000, workload=<Workload.fulltime: 'fulltime'>, worker_id=1, i

***JOIN***

In [22]:
with session_factory() as session:
    query = (
        select(
            ResumeOrm.workload,
            func.avg(ResumeOrm.compensation).cast(Integer).label("avg_compensation"),               
    )
    .select_from(ResumeOrm)
    .filter(and_(
        ResumeOrm.title.icontains("python"),
        ResumeOrm.compensation > 40000
    ))
    .group_by(ResumeOrm.workload)
    .having(func.avg(ResumeOrm.compensation) > 70000)
    )
    res = session.execute(query)
    result_orm = res.all()
    print(f"{result=}")


result=[(<Workload.parttime: 'parttime'>, 165000), (<Workload.fulltime: 'fulltime'>, 90000)]


In [24]:
from pydantic import BaseModel

class WorkloadAvgCompensationDTO(BaseModel):
    workload: Workload
    avg_compensation: int

result_dto = [WorkloadAvgCompensationDTO.model_validate(row, from_attributes=True) for row in result_orm]
print(f"{result_dto=}")

result_dto=[WorkloadAvgCompensationDTO(workload=<Workload.parttime: 'parttime'>, avg_compensation=165000), WorkloadAvgCompensationDTO(workload=<Workload.fulltime: 'fulltime'>, avg_compensation=90000)]


In [32]:
print(1 or 1 and print("hello world"))

1
